In [12]:
import itk
from itkwidgets import view, compare, checkerboard
from itkwidgets.widget_viewer import Viewer
import ipywidgets
from ipywidgets import Button, Label, HBox, VBox
import os

# these FileUpload widgets have maximum file size
# controlled via jupyter_notebook_config.py
# https://github.com/jupyter-widgets/ipywidgets/issues/2522
fixedUploader = ipywidgets.FileUpload(
    accept='.nrrd,.mha,.nii,image/*',
    multiple=False)
movingUploader = ipywidgets.FileUpload(
    accept='.nrrd,.mha,.nii,image/*',
    multiple=False)


vboxFixed = VBox([Label('Fixed image'), fixedUploader, Viewer()])
vboxMoving = VBox([Label('Moving image'), movingUploader, Viewer()])
vboxResult = VBox([Label('Result image'), Label(' '), Viewer()])
imageViews = HBox([vboxFixed, vboxMoving, vboxResult])
display(imageViews)

appState={'fixed':None, 'moving':None}
out = ipywidgets.Output()
display(out)

viewButton = ipywidgets.Button(description="Load and view")
regType=ipywidgets.RadioButtons(
    options=['rigid', 'affine', 'bspline'],
    value='rigid', # Defaults to 'pineapple'
    description='Type:',
    disabled=False
)
registerButton = ipywidgets.Button(description="Register and view")
buttons = HBox([viewButton, regType, registerButton])
display(buttons)


def viewImages(b):
    movingButton=Button(description='Moving Updated')
    
    # write to temporary image file
    # I don't know how to read image from memory in Python
    
    # using FileUpload's syntax for ipywidgets <= 7
    tempFilename="./fixed_"+ next(iter(fixedUploader.value))
    with open(tempFilename, "wb") as fp:
        fp.write(fixedUploader.data[0])
    appState['fixed'] = itk.imread(tempFilename, itk.F)
    os.remove(tempFilename)

    tempFilename="./moving_"+ next(iter(movingUploader.value))
    with open(tempFilename, "wb") as fp:
        fp.write(movingUploader.data[0])
    appState['moving'] = itk.imread(tempFilename, itk.F)
    os.remove(tempFilename)
    
    
    vboxFixed=imageViews.children[0]
    vboxFixed.children=(vboxFixed.children[0],
                        vboxFixed.children[1],
                        Viewer(image=appState['fixed']))
    
    vboxMoving=imageViews.children[1]
    vboxMoving.children=(vboxMoving.children[0],
                        vboxMoving.children[1],
                        Viewer(image=appState['moving']))
    
    vboxResult=imageViews.children[2]
    vboxResult.children=(Label('Fixed/moving'),
                        Label('checkerboard'),
                        checkerboard(appState['fixed'], appState['moving']))
    
    imageViews.children=(vboxFixed,
                        vboxMoving,
                        vboxResult)
    
    out.clear_output(wait=True)
    # with out:
#     compareWidget=compare(appState['fixed'],
#                           appState['moving'],
#                           link_cmap=True,
#                           link_gradient_opacity=True)

viewButton.on_click(viewImages)

def registerImages(b):
    out.clear_output(wait=True)
    parameters = itk.ParameterObject.New()

    resolutions = 3
    default_rigid = parameters.GetDefaultParameterMap("rigid", resolutions)
    parameters.AddParameterMap(default_rigid)

    if (regType.value!='rigid'):
        resolutions = 2
        default_affine = parameters.GetDefaultParameterMap("affine", resolutions)
        parameters.AddParameterMap(default_affine)

    if (regType.value=='bspline'):
        resolutions = 1
        default_bspline = parameters.GetDefaultParameterMap("bspline", resolutions)
        parameters.AddParameterMap(default_bspline)

    parameters.RemoveParameter("ResultImageFormat")
    with out:
        print('Executing '+regType.value+' registration. Please wait...')
    
    registered_moving = itk.elastix_registration_method(appState['fixed'], appState['moving'], parameter_object=parameters)
    
    vboxResult=imageViews.children[2]
    vboxResult.children=(Label('Fixed/registered'),
                        vboxResult.children[1],
                        checkerboard(appState['fixed'], registered_moving))
    
    imageViews.children=(imageViews.children[0],
                        imageViews.children[1],
                        vboxResult)
    
    # display(checkerboard(appState['fixed'], registered_moving))
    out.clear_output()

registerButton.on_click(registerImages)

Output()